In [ ]:
# ============================================
# IMPORTACIÓN DE LIBRERÍAS NECESARIAS
# ============================================

# Librerías para manipulación y análisis de datos
import pandas as pd  # Pandas: trabajar con DataFrames y series de datos
import numpy as np   # NumPy: operaciones matemáticas y arrays numéricos

# Librerías para visualización de datos
import matplotlib as plt  # Matplotlib: crear gráficos básicos
import seaborn as sns     # Seaborn: gráficos estadísticos más avanzados

# Librerías de scikit-learn para Machine Learning
from sklearn.model_selection import train_test_split  # Dividir datos en entrenamiento y test
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler  # Transformar features
from sklearn.linear_model import LinearRegression, Ridge, Lasso  # Modelos de regresión lineal
from sklearn.metrics import mean_absolute_error, mean_squared_error  # Métricas para evaluar modelos

In [ ]:
# ============================================
# CARGA DE DATOS DE LA COMPETICIÓN DE KAGGLE
# ============================================

# Cargamos el conjunto de entrenamiento (contiene la columna objetivo Price_euros)
df = pd.read_csv('./data/train.csv')

# Cargamos el conjunto de test (NO contiene Price_euros, debemos predecirlo)
test = pd.read_csv('./data/test.csv')

# Cargamos el archivo de ejemplo para saber el formato correcto del submission
sample = pd.read_csv('./data/sample_submission.csv')

In [ ]:
# ============================================
# PREPROCESAMIENTO: LIMPIEZA DE LA VARIABLE RAM
# ============================================

# La columna 'Ram' contiene valores como texto: "8GB", "16GB", "32GB"
# Los modelos de ML necesitan números, no texto

# Paso 1: Eliminamos el texto "GB" para quedarnos solo con el número
df['Ram'] = df['Ram'].str.replace('GB', '')

# Paso 2: Convertimos la columna a tipo numérico entero
df['Ram'] = df['Ram'].astype(int)

# Ahora Ram contiene: 8, 16, 32 en lugar de "8GB", "16GB", "32GB"

In [ ]:
# ============================================
# SELECCIÓN DE VARIABLES NUMÉRICAS
# ============================================

# Seleccionamos SOLO las columnas numéricas del DataFrame
# Los modelos de regresión trabajan con números, no con texto
dfn = df.select_dtypes(include=['number'])

# Las variables numéricas son: id, laptop_ID, Inches, Ram, Price_euros
# Las variables de texto como Company, Product, TypeName quedan excluidas
# Para usar variables de texto necesitaríamos técnicas de encoding

In [ ]:
# ============================================
# PREPARACIÓN DE VARIABLES X E Y
# ============================================

# X: Variables independientes (features o características)
# Son las variables que usaremos para predecir el precio
# Eliminamos Price_euros porque es lo que queremos predecir, no una característica
X = np.array(dfn.drop(columns=['Price_euros']))

# y: Variable dependiente (target u objetivo)
# Es lo que queremos predecir: el precio del portátil en euros
y = np.array(dfn['Price_euros'])

# Convertimos a arrays de NumPy para mayor eficiencia en scikit-learn

In [ ]:
# ============================================
# DIVISIÓN DE DATOS: ENTRENAMIENTO Y VALIDACIÓN
# ============================================

# Dividimos los datos en dos conjuntos:
# - Conjunto de ENTRENAMIENTO: para que el modelo aprenda patrones
# - Conjunto de TEST/VALIDACIÓN: para evaluar qué tan bien predice en datos nuevos

# Parámetros:
# - test_size=0.2: reservamos 20% para test, 80% para entrenamiento
# - random_state=42: semilla aleatoria para que la división sea reproducible
#   (siempre obtengamos la misma división cada vez que ejecutemos el código)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificamos las dimensiones de cada conjunto
print(X_train.shape)  # (729, 4) - 729 portátiles, 4 características
print(X_test.shape)   # (183, 4) - 183 portátiles, 4 características
print(y_train.shape)  # (729,) - 729 precios
print(y_test.shape)   # (183,) - 183 precios

In [ ]:
# ============================================
# ENTRENAMIENTO DEL MODELO: REGRESIÓN LINEAL
# ============================================

# Creamos el modelo de Regresión Lineal
# La regresión lineal busca encontrar la ecuación de una línea/hiperplano:
# y = b0 + b1*x1 + b2*x2 + ... + bn*xn
# donde b0, b1, b2... son los coeficientes que el modelo aprenderá
lr = LinearRegression()

# Entrenamos el modelo con los datos de entrenamiento
# El modelo ajusta los coeficientes para minimizar el error en las predicciones
lr.fit(X_train, y_train)

In [ ]:
# ============================================
# EVALUACIÓN DEL MODELO
# ============================================

# Hacemos predicciones sobre el conjunto de test (datos que el modelo NO ha visto)
y_pred = lr.predict(X_test)

# MÉTRICA 1: R² Score (Coeficiente de Determinación)
# - Rango: de 0 a 1 (puede ser negativo si el modelo es muy malo)
# - R² = 1: predicciones perfectas
# - R² = 0: el modelo predice igual que usar la media de precios
# - Interpretación: qué porcentaje de la variabilidad del precio es explicado por el modelo
print("R2 score", lr.score(X_test, y_test))

# MÉTRICA 2: MAE (Mean Absolute Error - Error Absoluto Medio)
# - Promedio de las diferencias absolutas entre predicción y valor real
# - Se interpreta en las mismas unidades que la variable objetivo (euros)
# - MAE = 322.29 significa que en promedio nos equivocamos por 322 euros
print("MAE", mean_absolute_error(y_test, y_pred))

In [ ]:
# ============================================
# VISUALIZACIÓN DEL DATASET DE TEST DE KAGGLE
# ============================================

# Este es el conjunto sobre el cual debemos hacer las predicciones finales
# NO contiene la columna Price_euros (es lo que debemos predecir y enviar a Kaggle)
test

In [ ]:
# ============================================
# PREPROCESAMIENTO DEL DATASET DE TEST
# ============================================

# MUY IMPORTANTE: Debemos aplicar exactamente las mismas transformaciones
# al conjunto de test que aplicamos al conjunto de entrenamiento

# Paso 1: Eliminamos "GB" de la columna Ram
test['Ram'] = test['Ram'].str.replace('GB', '')

# Paso 2: Convertimos a tipo entero
test['Ram'] = test['Ram'].astype(int)

# Si no hacemos esto, el modelo no podrá hacer predicciones correctas

In [ ]:
# ============================================
# SELECCIÓN DE VARIABLES NUMÉRICAS EN TEST
# ============================================

# Seleccionamos solo las columnas numéricas del test
# Esto debe coincidir exactamente con las columnas que usamos en el entrenamiento
test = test.select_dtypes(include=['number'])

In [ ]:
# ============================================
# PREDICCIONES SOBRE EL DATASET DE TEST
# ============================================

# Usamos el modelo ya entrenado para predecir los precios del conjunto de test
# Estas predicciones son las que enviaremos a Kaggle
prediction_test = lr.predict(test)

# Mostramos el array con todas las predicciones
prediction_test

In [ ]:
# ============================================
# AÑADIR PREDICCIONES AL DATAFRAME DE TEST
# ============================================

# Creamos una nueva columna 'Price_euros' con nuestras predicciones
# Ahora el DataFrame de test tiene los precios predichos
test['Price_euros'] = lr.predict(test)

In [ ]:
# ============================================
# PREPARACIÓN DEL ARCHIVO DE SUBMISSION
# ============================================

# Kaggle requiere un archivo CSV con formato específico:
# - Primera columna: 'id' (identificador único de cada portátil)
# - Segunda columna: 'Price_euros' (nuestras predicciones)
borja = test[['id', 'Price_euros']]

# Es importante revisar el archivo sample_submission.csv para confirmar el formato

In [ ]:
# ============================================
# VISUALIZACIÓN DEL DATAFRAME DE SUBMISSION
# ============================================

# Verificamos que el DataFrame tiene el formato correcto antes de exportar
# Debe tener 391 filas (una por cada portátil del test) y 2 columnas (id, Price_euros)
borja

In [ ]:
# ============================================
# EXPORTACIÓN DEL ARCHIVO DE SUBMISSION
# ============================================

# Guardamos el DataFrame como archivo CSV
# - index=False: no incluir el índice de pandas como columna adicional
# - header=True: incluir los nombres de las columnas (id, Price_euros)
borja.to_csv('borja.csv', index=False, header=True)

# PASOS SIGUIENTES PARA ENVIAR A KAGGLE:
# 1. Descargar el archivo 'borja.csv' generado
# 2. Ir a la página de la competición en Kaggle
# 3. Hacer clic en "Submit Predictions"
# 4. Subir el archivo CSV
# 5. Esperar a que Kaggle evalúe las predicciones y ver tu puntuación



In [ ]:
# ============================================
# RESUMEN DEL WORKFLOW DE KAGGLE
# ============================================

# Este notebook sigue el flujo de trabajo estándar de una competición de Kaggle:

# 1. CARGA DE DATOS
#    - Cargar train.csv, test.csv y sample_submission.csv

# 2. EXPLORACIÓN Y ANÁLISIS
#    - Visualizar datos con .head(), .info()
#    - Analizar correlaciones y distribuciones

# 3. PREPROCESAMIENTO
#    - Limpiar datos (eliminar texto innecesario, convertir tipos)
#    - Tratar valores faltantes si los hay

# 4. SELECCIÓN DE FEATURES
#    - Decidir qué variables usar en el modelo
#    - Realizar feature engineering si es necesario

# 5. DIVISIÓN TRAIN/TEST
#    - Separar datos para entrenamiento y validación
#    - Nunca entrenar con datos de test

# 6. ENTRENAMIENTO DEL MODELO
#    - Elegir un algoritmo (en este caso LinearRegression)
#    - Entrenar con los datos de entrenamiento

# 7. EVALUACIÓN
#    - Medir el rendimiento con métricas (R², MAE, RMSE)
#    - Validar que el modelo generaliza bien

# 8. PREDICCIONES
#    - Aplicar el modelo al conjunto de test
#    - Generar las predicciones finales

# 9. SUBMISSION
#    - Preparar el archivo en el formato correcto
#    - Enviar a Kaggle y revisar la puntuación

